У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

In [63]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [64]:
data_dir = '../../../data/'
train_path = data_dir + 'customer_segmentation_train.csv'

raw_df = pd.read_csv(train_path)
raw_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [65]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [66]:
raw_df.Segmentation.value_counts()

Segmentation
D    2268
A    1972
C    1970
B    1858
Name: count, dtype: int64

In [67]:
train_df, test_df = train_test_split(raw_df, test_size=0.2, random_state=42, stratify=raw_df.Segmentation)

input_cols = raw_df.columns[:-1].tolist()
target_col = raw_df.columns[-1]

train_inputs, train_targets = train_df[input_cols], train_df[target_col]
test_inputs, test_targets = test_df[input_cols], test_df[target_col]

numeric_cols = train_inputs.select_dtypes('number').columns.tolist()
categorical_cols = train_inputs.select_dtypes('object').columns.tolist()

In [68]:
numeric_cols, categorical_cols

(['ID', 'Age', 'Work_Experience', 'Family_Size'],
 ['Gender',
  'Ever_Married',
  'Graduated',
  'Profession',
  'Spending_Score',
  'Var_1'])

In [69]:
raw_df.Gender.unique()

array(['Male', 'Female'], dtype=object)

In [70]:
raw_df.Ever_Married.unique()

array(['No', 'Yes', nan], dtype=object)

In [71]:
raw_df.Graduated.unique()

array(['No', 'Yes', nan], dtype=object)

In [72]:
raw_df.Profession.unique()

array(['Healthcare', 'Engineer', 'Lawyer', 'Entertainment', 'Artist',
       'Executive', 'Doctor', 'Homemaker', 'Marketing', nan], dtype=object)

In [73]:
raw_df.Spending_Score.unique()

array(['Low', 'Average', 'High'], dtype=object)

In [74]:
raw_df.Var_1.unique()

array(['Cat_4', 'Cat_6', 'Cat_7', 'Cat_3', 'Cat_1', 'Cat_2', nan, 'Cat_5'],
      dtype=object)

In [75]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler()),
    # ('scaler', StandardScaler())

])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

train_inputs_values = preprocessor.fit_transform(train_inputs)
test_inputs_values = preprocessor.transform(test_inputs) 


transformed_cols = preprocessor['cat']['onehot'].get_feature_names_out(categorical_cols).tolist()

train_X = pd.DataFrame(train_inputs_values, columns=numeric_cols + transformed_cols)  
test_X = pd.DataFrame(test_inputs_values, columns=numeric_cols + transformed_cols)  


In [76]:
test_X.head()

,ID,Age,Work_Experience,Family_Size,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Ever_Married_missing,Graduated_No,...,Spending_Score_High,Spending_Score_Low,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7,Var_1_missing
0,0.401023,0.323944,0.857143,0.250,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.087300,0.042254,0.285714,0.375,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.499666,0.253521,0.000000,0.000,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.779693,0.126761,0.000000,0.125,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.003781,0.000000,0.000000,0.625,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [77]:
train_X.shape

(6454, 33)

In [78]:
from imblearn.over_sampling import SMOTE, SMOTENC

smote = SMOTE(random_state=247)
X_train_smote, Y_train_smote = smote.fit_resample(train_X[numeric_cols], train_targets)

X_train_smote.shape

(7256, 4)

In [79]:
transformed_cols_ids = [col_id for col_id, col_name in enumerate(train_X.columns) if col_name in transformed_cols]

smotenc = SMOTENC(random_state=247, categorical_features=transformed_cols_ids)
X_train_smotenc, Y_train_smotenc = smotenc.fit_resample(train_X, train_targets)

X_train_smotenc.shape

(7256, 33)

In [80]:
from imblearn.combine import SMOTETomek

smote_tomek = SMOTETomek(random_state=247)
X_train_smote_tomek, Y_train_smote_tomek = smote_tomek.fit_resample(train_X, train_targets)

X_train_smote_tomek.shape

(5552, 33)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [81]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


log_reg = LogisticRegression(solver='liblinear')

ovr_clf = OneVsRestClassifier(log_reg)

In [82]:
def train_predict(model, X_train, Y_train, X_test=test_X, Y_test=test_targets):
    
    model.fit(X_train, Y_train)

    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)

    print("Тренувальні данні")
    print(classification_report(Y_train, predictions_train))
    print("---------------------------------------------------")
    print("Тестові данні")
    print(classification_report(Y_test, predictions_test))


In [83]:
res = train_predict(ovr_clf, train_X, train_targets)

Тренувальні данні
              precision    recall  f1-score   support

           A       0.43      0.50      0.46      1578
           B       0.41      0.17      0.24      1486
           C       0.51      0.63      0.56      1576
           D       0.64      0.72      0.68      1814

    accuracy                           0.52      6454
   macro avg       0.50      0.51      0.49      6454
weighted avg       0.50      0.52      0.50      6454

---------------------------------------------------
Тестові данні
              precision    recall  f1-score   support

           A       0.43      0.47      0.45       394
           B       0.41      0.17      0.24       372
           C       0.49      0.62      0.55       394
           D       0.65      0.75      0.70       454

    accuracy                           0.52      1614
   macro avg       0.49      0.50      0.48      1614
weighted avg       0.50      0.52      0.49      1614



In [84]:
res_smote = train_predict(ovr_clf, X_train_smote, Y_train_smote, test_X[numeric_cols], test_targets)

Тренувальні данні
              precision    recall  f1-score   support

           A       0.34      0.30      0.32      1814
           B       0.31      0.11      0.16      1814
           C       0.39      0.47      0.42      1814
           D       0.47      0.73      0.57      1814

    accuracy                           0.40      7256
   macro avg       0.38      0.40      0.37      7256
weighted avg       0.38      0.40      0.37      7256

---------------------------------------------------
Тестові данні
              precision    recall  f1-score   support

           A       0.34      0.27      0.30       394
           B       0.23      0.08      0.12       372
           C       0.37      0.44      0.40       394
           D       0.49      0.76      0.60       454

    accuracy                           0.41      1614
   macro avg       0.36      0.39      0.36      1614
weighted avg       0.37      0.41      0.37      1614



In [85]:
res_smotenc = train_predict(ovr_clf, X_train_smotenc, Y_train_smotenc)

Тренувальні данні
              precision    recall  f1-score   support

           A       0.43      0.50      0.46      1814
           B       0.42      0.25      0.31      1814
           C       0.51      0.60      0.55      1814
           D       0.64      0.68      0.66      1814

    accuracy                           0.51      7256
   macro avg       0.50      0.51      0.50      7256
weighted avg       0.50      0.51      0.50      7256

---------------------------------------------------
Тестові данні
              precision    recall  f1-score   support

           A       0.42      0.47      0.44       394
           B       0.37      0.23      0.28       372
           C       0.50      0.57      0.53       394
           D       0.67      0.72      0.69       454

    accuracy                           0.51      1614
   macro avg       0.49      0.50      0.49      1614
weighted avg       0.50      0.51      0.50      1614



In [86]:
res_smote_tomec = train_predict(ovr_clf, X_train_smote_tomek, Y_train_smote_tomek)

Тренувальні данні
              precision    recall  f1-score   support

           A       0.47      0.55      0.51      1349
           B       0.44      0.26      0.33      1359
           C       0.55      0.65      0.60      1412
           D       0.69      0.72      0.71      1432

    accuracy                           0.55      5552
   macro avg       0.54      0.55      0.53      5552
weighted avg       0.54      0.55      0.54      5552

---------------------------------------------------
Тестові данні
              precision    recall  f1-score   support

           A       0.42      0.50      0.45       394
           B       0.35      0.21      0.26       372
           C       0.50      0.58      0.54       394
           D       0.68      0.70      0.69       454

    accuracy                           0.51      1614
   macro avg       0.49      0.50      0.49      1614
weighted avg       0.50      0.51      0.50      1614



Для оцінки якості моделей оберемо `macro avg f1-score`. 

Найгірше значення матрики у моделі навченої на збаланасованому методом `SMOTE` датасеті з без категоріальних ознак. Очевидно, це зумовлено тим, що у моделі замало ознак для виявлення залежності.

Іші моделі мають приблизно однакові значення, 0.48 для незбалансованих даних та 0.49 для збалансованих методами `SMOTENC` та `SmoteTomek` на тестових даних. Між моделями немає значної різниці скоріщ за все через те, що початкова різниця між кількістю ексземплярів різних класів не була дуже великою